In [7]:
# import libraries
import os
import pandas as pd
import sqlite3
import sys

# Add the WSJ directory to sys.path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# import text_cleaner
from text_cleaner_WSJ import clean_article_text

In [ ]:
# connect to the SQLite database and read the data into a DataFrame
db_path = "articlesWSJ.db"
conn = sqlite3.connect(db_path)
df = pd.read_sql_query("SELECT * FROM articles_index", conn)
conn.close()

Based on the literature review and exploratory analysis (see Exposé), several article categories were identified as irrelevant to the research objective. These are excluded in the following section.

In [ ]:
# extract link column  
articles_links = df['link']

# extract the section names from the links
articles_sections = [x[19:40] for x in articles_links]
articles_sections = [x.split('/')[1] for x in articles_sections]

# add section names to the DataFrame
df['section'] = articles_sections

# extract unique section names
unique_sections = set(articles_sections)
print(unique_sections)

# create list of irrelevant sections for later removal
irrelevant_sections = ["health","arts-culture","lifestyle","real-estate","sports","livecoverage","personal-finance","video","science","style","articles"]

# investigate headlines
print(df[df["headline"].duplicated()])  # Check for duplicates in headlines
display(df[df["headline"].duplicated()])  # Display duplicates in headlines


In [ ]:
# remove irrelevant sections from the DataFrame
df_filtered = df[~df['section'].isin(irrelevant_sections)].copy()

# remove salt and pepper noise from the headlines as manual investigation showed that they are not relevant for the analysis (only comics)
df_filtered = df_filtered[~df_filtered['headline'].str.contains(r'\b(salt|pepper)\b', case=False, na=False)]

# verify sections value counts after filtering
print(df_filtered['section'].value_counts())  # See what's left

# verify drop of duplicates in headlines
print(df_filtered[df_filtered["headline"].duplicated()])  

In [ ]:
# check for misssing values
print(df_filtered.isnull().sum())  # Check for missing values

In [ ]:
# Define path for new DB
cleaned_db_path = r"articles_index_cleaned_2024.db"

# Save DataFrame to a new SQLite database
conn = sqlite3.connect(cleaned_db_path)
df_filtered.to_sql("articles_index_cleaned_2024", conn, if_exists="replace", index=False)
conn.close()

The article_ids from the cleaned index table are joined with the original articles table (containing the full text) to retain only relevant articles with available corpora. The original database is preserved to ensure reproducibility.

In [ ]:
# Paths to databases
cleaned_index_path = r"articles_index_cleaned_2024.db"
original_db_path = r"articlesWSJ_2024.db"
final_db_path = r"articlesWSJ_clean_final_2024.db"

# Load the cleaned articles_index table
print("Loading cleaned articles_index from cleaned_index_path...")
conn_cleaned = sqlite3.connect(cleaned_index_path)
df_cleaned_index = pd.read_sql_query("SELECT * FROM articles_index_cleaned_2024", conn_cleaned)
conn_cleaned.close()
print(f"Loaded {len(df_cleaned_index)} cleaned index entries.")

# Write the cleaned index into the original DB temporarily
print("Attaching cleaned index to original database...")
conn_full = sqlite3.connect(original_db_path)
df_cleaned_index.to_sql("articles_index_cleaned_2024", conn_full, if_exists="replace", index=False)

# Perform the join to filter articles based on valid index_ids
print("Joining article table with cleaned index on index_id...")
query = """
SELECT article.*, articles_index_cleaned_2024.section, articles_index_cleaned_2024.year, articles_index_cleaned_2024.month, articles_index_cleaned_2024.day
FROM article
JOIN articlesWSJ_2024
ON article.index_id = articlesWSJ_2024.id
"""
df_filtered_articles = pd.read_sql_query(query, conn_full)
conn_full.close()
print(f"Filtered down to {len(df_filtered_articles)} articles.")

# Save cleaned articles and index into final DB
print("Saving filtered article and cleaned articles_index into final database...")
conn_final = sqlite3.connect(final_db_path)
df_filtered_articles.to_sql("article", conn_final, if_exists="replace", index=False)
df_cleaned_index.to_sql("articles_index", conn_final, if_exists="replace", index=False)
conn_final.close()

print("Final cleaned database successfully created.")

In [ ]:
# load data from new database to verify
final_db_path = r"articlesWSJ_clean_final_2024.db"
conn = sqlite3.connect(final_db_path)
df = pd.read_sql_query("SELECT * FROM article", conn)
conn.close()

In [ ]:
# find duplicats in corpus
duplicates = df[df.duplicated(subset=['corpus'], keep=False)]
print("Duplicated corpus rows:\n", duplicates[['corpus', 'article_id']].head(10))
print("Number of duplicated corpus rows:", len(duplicates))

# drop duplicates
df = df.drop_duplicates(subset=['corpus'], keep='first')   

# verify that duplicates are removed
duplicates_after = df[df.duplicated(subset=['corpus'], keep=False)] 
print("Duplicated corpus rows after dropping duplicates:\n", duplicates_after[['corpus', 'article_id']].head(10))

In [ ]:
# drop image_src
df = df.drop(columns=['image_src'])

In [ ]:
# Drop rows where article_id is NULL
df_no_na = df.dropna(subset=["article_id"])

# verify drop
print("Number of rows after dropping rows with NULL article_id:", len(df_no_na[df_no_na['article_id'].isnull()]))

# check for any duplicates in the 'article_id' column
duplicates_article_id = df_no_na[df_no_na.duplicated(subset=['article_id'], keep=False)]
print("Duplicated article_id rows:\n", duplicates_article_id[['article_id', 'corpus']].head(10))

# check for duplicates in corpus column
duplicates_corpus = df_no_na[df_no_na.duplicated(subset=['corpus'], keep=False)]
print("Duplicated corpus rows:\n", duplicates_corpus[['corpus', 'article_id']].head(10))

# check for missing values in the 'corpus' column
missing_corpus = df_no_na[(df_no_na['corpus'].isnull()) | (df_no_na['corpus'] == '')]
print("Missing corpus rows:\n", missing_corpus[['article_id', 'corpus']].head(10))

In [ ]:
# merching year, month, day into a single date column
df_no_na['date'] = pd.to_datetime(df_no_na[['year', 'month', 'day']].astype(str).agg('-'.join, axis=1), format='%Y-%m-%d')

# verify the new date column
print("Date column:\n", df_no_na[['year', 'month', 'day', 'date']].head(10))
print("Date column data type:", df_no_na['date'].dtype)

# drop the old columns
df_no_na = df_no_na.drop(columns=['year', 'month', 'day'])

In [ ]:
# use cleaner function to clean the corpus
cleaned_df = df_no_na.copy()
cleaned_df['cleaned_corpus'] = df_no_na['corpus'].apply(clean_article_text)

In [ ]:
# verify corpus cleaning
print("Cleaned corpus:\n", cleaned_df[['article_id', 'cleaned_corpus']].head(10))
print("Number of cleaned corpus rows:", len(cleaned_df))

In [ ]:
# overrite the original DataFrame with the cleaned corpus
df_no_na['corpus'] = cleaned_df['cleaned_corpus']

# drop # drop image_src
df_no_na = df_no_na.drop(columns=['image_src'])

In [ ]:
# Temporarily expand column width and disable truncation
pd.set_option('display.max_colwidth', None)

# Inspect full cleaned corpus
print("Cleaned corpus:\n", df_no_na['corpus'].iloc[:10])
print("column names:\n", df_no_na.columns)
print("DataFrame info:\n", df_no_na.info())

In [ ]:
# connect to the final database
final_db_path = r"articlesWSJ_clean_final_2024.db"
conn = sqlite3.connect(final_db_path)

# Write the cleaned DataFrame to the 'article' table
df_no_na.to_sql('article', conn, if_exists='replace', index=False)

# Close the connection
conn.close()

In [ ]:
# verify that the corpus is cleaned
conn = sqlite3.connect(final_db_path)
df = pd.read_sql_query("SELECT * FROM article", conn)
conn.close()

# column names of the DataFrame 
print("Column names in the DataFrame:\n", df.columns)

# cleaned corpus
print("Cleaned corpus:\n", df['corpus'].head(10))

# check for missing values in the DataFrame
print("Missing values in the DataFrame:\n", df.isnull().sum())

# number of rows in the DataFrame
print("Number of rows in the DataFrame:", len(df))